# Install packages

In [5]:
!pip install hopsworks dotenv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


# Import packages

In [4]:
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import requests
import hopsworks
import os
os.environ['EIA_API_KEY'] = "4goTb0VNLmTBcF4hd6VyD1BHsJNcgA6ufRW4R6ax"
#from dotenv import load_dotenv
#%load_ext dotenv

ModuleNotFoundError: ignored

# Connect to GDrive

In [14]:
from google.colab import drive

drive.mount("/content/gdrive")
os.chdir("gdrive/MyDrive/Serverless/ElectricityDemand/")

Mounted at /content/gdrive


# Connect to API

In [7]:
url = ('https://api.eia.gov/v2/electricity/rto/daily-region-data/data/'
       '?frequency=daily'
       '&data[0]=value'
       '&facets[respondent][]=NY'
       '&facets[timezone][]=Eastern'
       '&facets[type][]=D'
       '&start=2017-01-01'
       '&end=2021-12-31'
       '&sort[0][column]=period'
       '&sort[0][direction]=desc'
       '&offset=0'
       '&length=5000'
       '&api_key=') + os.environ.get('EIA_API_KEY')

In [8]:
data = requests.get(url).json()['response']['data']
data = pd.DataFrame(data)

In [9]:
data.head(5)

,period,respondent,respondent-name,type,type-name,timezone,timezone-description,value,value-units
0,2021-12-31,NY,New York,D,Demand,Eastern,Eastern,380279,megawatthours
1,2021-12-30,NY,New York,D,Demand,Eastern,Eastern,401376,megawatthours
2,2021-12-29,NY,New York,D,Demand,Eastern,Eastern,411729,megawatthours
3,2021-12-28,NY,New York,D,Demand,Eastern,Eastern,411471,megawatthours
4,2021-12-27,NY,New York,D,Demand,Eastern,Eastern,424491,megawatthours


In [10]:
data.dtypes

period                  object
respondent              object
respondent-name         object
type                    object
type-name               object
timezone                object
timezone-description    object
value                    int64
value-units             object
dtype: object

# Create the Demand dataset

In [28]:
daily_demand = data[["period", "value"]].rename(columns={"period":"date", "value": "demand"})
daily_demand["date"] = pd.to_datetime(daily_demand["date"], infer_datetime_format=True)
daily_demand.head()

,date,demand
0,2021-12-31,380279
1,2021-12-30,401376
2,2021-12-29,411729
3,2021-12-28,411471
4,2021-12-27,424491


# Get weather data

In [24]:
!wget https://raw.githubusercontent.com/aykhazanchi/id2223-scalable-ml/master/proj/noaa_weather_lag_2017-2021.csv "ElectricityDemand/"

--2023-03-01 23:17:13--  https://raw.githubusercontent.com/aykhazanchi/id2223-scalable-ml/master/proj/noaa_weather_lag_2017-2021.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 110845 (108K) [text/plain]
Saving to: ‘noaa_weather_lag_2017-2021.csv.3’

noaa_weather_lag_20 100%[===================>] 108.25K  --.-KB/s    in 0.01s   

2023-03-01 23:17:13 (7.74 MB/s) - ‘noaa_weather_lag_2017-2021.csv.3’ saved [110845/110845]

--2023-03-01 23:17:13--  http://electricitydemand/
Resolving electricitydemand (electricitydemand)... failed: Name or service not known.
wget: unable to resolve host address ‘electricitydemand’
FINISHED --2023-03-01 23:17:13--
Total wall clock time: 0.1s
Downloaded: 1 files, 108K in 0.01s (7.74 MB/s)


In [29]:
!ls

feature.ipynb  noaa_weather_lag_2017-2021.csv


In [32]:
weather_df = pd.read_csv('noaa_weather_lag_2017-2021.csv')
weather_df.head()

,STATION,NAME,DATE,TAVG
0,USW00014732,"LAGUARDIA AIRPORT, NY US",2017-01-01,8.0
1,USW00014732,"LAGUARDIA AIRPORT, NY US",2017-01-02,4.4
2,USW00014732,"LAGUARDIA AIRPORT, NY US",2017-01-03,5.6
3,USW00014732,"LAGUARDIA AIRPORT, NY US",2017-01-04,8.2
4,USW00014732,"LAGUARDIA AIRPORT, NY US",2017-01-05,1.0


# Create time series data from weather data

In [33]:
daily_weather = weather_df[["DATE", "TAVG"]].rename(columns={"DATE": "date", "TAVG": "temperature"})
daily_weather['date'] = pd.to_datetime(daily_weather['date'], infer_datetime_format=True)
daily_weather.head()

,date,temperature
0,2017-01-01,8.0
1,2017-01-02,4.4
2,2017-01-03,5.6
3,2017-01-04,8.2
4,2017-01-05,1.0


In [34]:
daily_weather.dtypes

date           datetime64[ns]
temperature           float64
dtype: object

# Merge daily demand and daily weather DF

In [35]:
df = pd.merge(daily_weather, daily_demand, how='inner', on='date')
df.head()

,date,temperature,demand
0,2017-01-01,8.0,385445
1,2017-01-02,4.4,417277
2,2017-01-03,5.6,434018
3,2017-01-04,8.2,430935
4,2017-01-05,1.0,459619


# Feature Engineering

## Feature Engineering - day of week

In [36]:
df['day'] = df['date'].dt.dayofweek
df.head()

,date,temperature,demand,day
0,2017-01-01,8.0,385445,6
1,2017-01-02,4.4,417277,0
2,2017-01-03,5.6,434018,1
3,2017-01-04,8.2,430935,2
4,2017-01-05,1.0,459619,3


## Feature Engineering - month

In [37]:
df['month'] = df['date'].dt.month
df.head()

,date,temperature,demand,day,month
0,2017-01-01,8.0,385445,6,1
1,2017-01-02,4.4,417277,0,1
2,2017-01-03,5.6,434018,1,1
3,2017-01-04,8.2,430935,2,1
4,2017-01-05,1.0,459619,3,1


## Feature Engineering - bank holiday

In [38]:
# bank holiday or not - 1 or 0
holidays = calendar().holidays(start=df['date'].min(), end=df['date'].max())
df['holiday'] = df['date'].isin(holidays).astype(int)
df.head()

,date,temperature,demand,day,month,holiday
0,2017-01-01,8.0,385445,6,1,0
1,2017-01-02,4.4,417277,0,1,1
2,2017-01-03,5.6,434018,1,1,0
3,2017-01-04,8.2,430935,2,1,0
4,2017-01-05,1.0,459619,3,1,0


# Hopsworks - Feature Store

In [39]:
# Login to hopsworks
project = hopsworks.login()
fs = project.get_feature_store()

Copy your Api Key (first register/login): https://c.app.hopsworks.ai/account/api/generated

Paste it here: ··········
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/3355
Connected. Call `.close()` to terminate connection gracefully.


In [40]:
# Create feature group
fg = fs.get_or_create_feature_group(name="ny_elec",
                                    primary_key=["date", "temperature", "day", "month", "holiday"],
                                    version = 1,
                                    description="NY electricity demand data with average daily weather and temperature")

In [41]:
# Insert feature group to hopsworks
fg.insert(df, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/3355/fs/3302/fg/21934


Uploading Dataframe: 0.00% |          | Rows 0/1826 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/3355/jobs/named/ny_elec_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7f1be5c9c460>, None)